In [352]:
import Rust_Machine_Learning
import plotly.express as px
import pandas as pd

In [353]:
# Get the csv data
df = pd.read_csv("../data/iris.data")
# df = pd.read_csv("../data/kmeans-dataset.csv")
# df = pd.read_csv("../data/winequality-red.csv")
# df = pd.DataFrame(
#     data={
#         "Loves Popcorn": [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0],
#         "Loves Soda": [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0],
#         "Age": [7.0, 12.0, 18.0, 35.0, 38.0, 50.0, 83.0],
#         "Loves Cool As Ice": [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0],
#     }
# )
df  


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [354]:
# Ensure categoriical data is one-hot encoded
# columns is list of ints
def one_hot_encode(df, columns):
    encoded = df.copy()
    feature_to_cols = dict()
    for col in columns:
        # Get the unique values
        unique_vals = encoded.iloc[:, col].unique()

        # Start col
        start_col = encoded.shape[1]-1

        # Now for each unique value add a col to df
        for val in unique_vals:
            col_encoded = encoded.iloc[:, col] == val
            col_encoded = [(1.0 if label == True else 0.0) for label in col_encoded]
            encoded["is_" + encoded.columns[col] + "_" + str(val)] = col_encoded
        
        end_col = encoded.shape[1]-1
        feature_to_cols[col] = list(range(start_col, end_col))

    encoded.drop(encoded.columns[columns], axis=1, inplace=True)
    return (encoded, feature_to_cols)


In [355]:
to_encode = [4]
encoded, feature_to_encoded_cols = one_hot_encode(df, to_encode)
encoded

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [356]:
# Split the data into train and test
def split_train_test(df, percent_train, randomize = True):
  #Randomize the rows
  if randomize:
    df = df.sample(frac=1.0)

  #Create train and test data
  df_size = df.shape[0]
  train_size = df_size * percent_train

  train_data = df.iloc[:int(train_size),:]
  test_data = df.iloc[int(train_size):,:]
  return (train_data, test_data)

In [357]:
# Get categorical based on indices
def get_is_categorical(categorical_original, df_original, encoded):
  is_categorical = []

  all_categories = categorical_original
  if(df_original.shape[1] != encoded.shape[1]):
    all_categories.extend(range(df_original.shape[1], encoded.shape[1]))

  for i in range(encoded.shape[1]):
    is_categorical.append(True if i in all_categories else False)
  return is_categorical

In [358]:
def remove_encoded_category_data_and_label(encoded, feature_to_encoded_cols, label_col):
  cols_to_remove = []
  did_remove_label_col = False
  for k,v in feature_to_encoded_cols.items():
    if label_col in v:
      encoded.drop(encoded.columns[v], axis=1, inplace=True)
      did_remove_label_col = True
  if not did_remove_label_col:
    encoded.drop(encoded.columns[label_col], axis=1, inplace=True)
    

In [359]:
# Prepare data for training with input rust functions expect
label_col = 6

train_data, test_data = split_train_test(encoded, 0.2, True)

# Remove any encoded data of the same class
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)
train_data_features_list = train_data_features.values.tolist()

# Get labels
train_data_labels = train_data.iloc[:, label_col].values.tolist()

# Determine which columns in train_data_features_list are categorical
is_categorical = [False, False, False, False]

In [360]:
train_data_features

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
128,6.4,2.8,5.6,2.1
81,5.5,2.4,3.7,1.0
17,5.1,3.5,1.4,0.3
15,5.7,4.4,1.5,0.4
12,4.8,3.0,1.4,0.1
6,4.6,3.4,1.4,0.3
80,5.5,2.4,3.8,1.1
73,6.1,2.8,4.7,1.2
83,6.0,2.7,5.1,1.6
144,6.7,3.3,5.7,2.5


In [361]:
# Now train the model
trained_model = Rust_Machine_Learning.DecisionTree(
    train_data_features_list, is_categorical, train_data_labels
)

#Note how it cheats if you leave in encoded data from same category.
# It will use what it is not to find what the label should be.
# Therefore data cleaning should include removing the one-hot-encoded data of same features
trained_model.print()

false 2 5.35
  0
  1


In [362]:
# Now let us test the model
test_data_features = test_data.copy()
remove_encoded_category_data_and_label(test_data_features, feature_to_encoded_cols, label_col)
test_data_features = test_data_features.values.tolist()
test_data_labels = test_data[test_data.columns[label_col]].values.tolist()

labelled_results = trained_model.classify(test_data_features)

num_correct = 0
for test_label, result_label in zip(test_data_labels, labelled_results):
  num_correct += test_label==result_label
print("Percent Correct: ", num_correct / len(test_data_labels))

Percent Correct:  0.8333333333333334
